In [1]:
import Pkg; Pkg.add(Pkg.PackageSpec(url="https://github.com/JuliaComputing/JuliaAcademyData.jl"))
using JuliaAcademyData; activate("Parallel_Computing")

    Cloning git-repo `https://github.com/JuliaComputing/JuliaAcademyData.jl`


g: [========================================>]  100.0 %Fetching: [=========>                               ]  22.0 %Fetching: [=========>                               ]  22.4 %Fetching: [===========>                             ]  26.9 %Fetching: [==============>                          ]  34.3 %Fetching: [=========================>               ]  61.2 %Fetching: [=========================>               ]  62.4 %Fetching: [==========================>              ]  64.1 %Fetching: [===========================>             ]  65.3 %Fetching: [===========================>             ]  66.9 %Fetching: [============================>            ]  68.6 %Fetching: [=============================>           ]  70.2 %Fetching: [=============================>           ]  71.8 %Fetching: [==============================>          ]  73.5 %Fetching: [===============================>         ]  75.1 %Fetching: [===============================>         ]  76.3 %Fetching: [===================

   Updating git-repo `https://github.com/JuliaComputing/JuliaAcademyData.jl`


   Updating registry at `~/.julia/registries/General`
   Updating registry at `~/.julia/registries/JuliaComputingRegistry`
######################################################################### 100.0%
  Resolving package versions...
  Installed LibVPX_jll ─────────────────── v1.9.0+1
  Installed FFMPEG_jll ─────────────────── v4.3.1+4
  Installed libvorbis_jll ──────────────── v1.3.6+6
  Installed Ogg_jll ────────────────────── v1.3.4+2
  Installed DataAPI ────────────────────── v1.4.0
  Installed Observables ────────────────── v0.3.2
  Installed GeometryBasics ─────────────── v0.3.3
  Installed StatsBase ──────────────────── v0.33.2
  Installed Requires ───────────────────── v1.1.0
  Installed x265_jll ───────────────────── v3.0.0+3
  Installed Tables ─────────────────────── v1.2.1
  Installed OpenSpecFun_jll ────────────── v0.5.3+4
  Installed libfdk_aac_jll ─────────────── v0.1.6+4
  Installed Compat ─────────────────────── v3.23.0
  Installed Distances ──────────────────── v0.9.

  Installed Compose ──────────────── v0.8.0
  Installed CUDAapi ──────────────── v2.1.0
  Installed NaNMath ──────────────── v0.3.3
  Installed OffsetArrays ─────────── v1.0.2
  Installed DataAPI ──────────────── v1.1.0
  Installed DiffEqDiffTools ──────── v1.6.0
  Installed WoodburyMatrices ─────── v0.5.0
  Installed Requires ─────────────── v0.5.2
  Installed Juno ─────────────────── v0.7.2
  Installed Distributions ────────── v0.22.3
  Installed Parameters ───────────── v0.12.0
  Installed FixedPointNumbers ────── v0.6.1
  Installed Distances ────────────── v0.8.2
  Installed AbstractTrees ────────── v0.3.1
  Installed CommonSubexpressions ─── v0.2.0
  Installed Compat ───────────────── v2.2.0
  Installed CuArrays ─────────────── v1.6.0
  Installed Zygote ───────────────── v0.4.6
  Installed Loess ────────────────── v0.5.0
  Installed Conda ────────────────── v1.3.0
  Installed ArrayInterface ───────── v2.2.0
  Installed DiffResults ──────────── v1.0.2
  Installed DocStringExtension

┌ Error: Error building `FFTW`: 
│ ERROR: LoadError: ArgumentError: Package Pkg not found in current path:
│ - Run `import Pkg; Pkg.add("Pkg")` to install the Pkg package.
│ 
│ Stacktrace:
│  [1] require(::Module, ::Symbol) at ./loading.jl:893
│  [2] include(::Function, ::Module, ::String) at ./Base.jl:380
│  [3] include at ./Base.jl:368 [inlined]
│  [4] load_julia_startup() at ./client.jl:324
│  [5] exec_options(::Base.JLOptions) at ./client.jl:267
│  [6] _start() at ./client.jl:506
│ in expression starting at /Applications/JuliaPro-1.5.1-1.app/Contents/Resources/julia/Contents/Resources/julia/etc/julia/startup.jl:4
└ @ Pkg.Operations /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Pkg/src/Operations.jl:949
   Building SpecialFunctions → `~/.julia/packages/SpecialFunctions/ne2iw/deps/build.log`
┌ Error: Error building `SpecialFunctions`: 
│ ERROR: LoadError: ArgumentError: Package Pkg not found in current path:
│ - Run `import Pkg; Pkg.add("Pkg")` to ins

# Outline of this notebook

- Define the sum function
- Implementations & benchmarking of sum in...
    - Julia (built-in)
    - Julia (hand-written)
    - C (hand-written)
    - python (built-in)
    - python (numpy)
    - python (hand-written)
- Towards exploiting parallelism with Julia
    - Allowing for floating point associativity
    - Making use of four cores at once: built-in
    - Making use of four cores at once: hand-written
- Summary of benchmarks

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [ ]:
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)

In [ ]:
sum(a)

The expected result is ~0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

In [ ]:
@time sum(a)

In [ ]:
@time sum(a)

In [ ]:
@time sum(a)

The `@time` macro can yield noisy results, so it's not our best choice for benchmarking!

Luckily, Julia has a `BenchmarkTools.jl` package to make benchmarking easy and accurate:

In [ ]:
using BenchmarkTools

In [ ]:
@benchmark sum($a)

# 1. Julia Built-in

So that's the performance of Julia's built-in sum — but that could be doing any number of tricks to be fast, including not using Julia at all in the first place! Of course, it is indeed written in Julia, but would it perform if we write a naive implementation ourselves?

In [ ]:
@which sum(a)

Let's save these benchmark results to a dictionary so we can start keeping track of them and comparing them down the line.

In [ ]:
j_bench = @benchmark sum($a)

In [ ]:
d = Dict()
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

# 2. Julia (hand-written)

In [ ]:
function mysum(A)
    s = 0.0
    for a in A
        s += a
    end
    return s
end

In [ ]:
j_bench_hand = @benchmark mysum($a)

In [ ]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

So that's about 2x slower than the builtin definition. We'll see why later on.

But first: is this fast?  How would we know?  Let's compare it to some other languages...

#  3. The C language

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

The current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it. Note that the `"""` wrap a multi-line string.

In [2]:
using Libdl
C_code = """
    #include <stddef.h>
    double c_sum(size_t n, double *X) {
        double s = 0.0;
        for (size_t i = 0; i < n; ++i) {
            s += X[i];
        }
        return s;
    }
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code)
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [ ]:
c_sum(a)

In [ ]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

We can now benchmark the C code directly from Julia:

In [ ]:
c_bench = @benchmark c_sum($a)

In [ ]:
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

# 4. Python's built in `sum`

The `PyCall` package provides a Julia interface to Python:

In [ ]:
using PyCall

In [ ]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

In [ ]:
pysum(a)

In [ ]:
pysum(a) ≈ sum(a)

In [ ]:
py_list_bench = @benchmark $pysum($a)

In [ ]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

# 5. Python: `numpy`

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [ ]:
using Conda

In [ ]:
# Conda.add("numpy")

In [ ]:
numpy_sum = pyimport("numpy")["sum"]

py_numpy_bench = @benchmark $numpy_sum($a)

In [ ]:
numpy_sum(a)

In [ ]:
numpy_sum(a) ≈ sum(a)

In [ ]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

# 6. Python, hand-written

In [ ]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

In [ ]:
py_hand = @benchmark $sum_py($a)

In [ ]:
sum_py(a)

In [ ]:
sum_py(a) ≈ sum(a)

In [ ]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

# Summary so far

In [ ]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

We seem to have three different performance classes here: The numpy and Julia
builtins are leading the pack, followed by the hand-written Julia and C
definitions. Those seem to be about 2x slower.  And then we have the much much
slower Python definitions over 100x slower.

# Exploiting parallelism with Julia

The fact that our hand-written Julia solution was almost an even multiple of
2x slower than the builtin solutions is a big clue: perhaps theres some sort
of 2x parallelism going on here?

(In fairness, there are ways to exploit parallelism in other languages, too,
but for brevity we won't cover them)

# 7. Julia (allowing floating point associativity)

The `for` loop

```julia
for a in A
    s += a
end
```

defines a very strict _order_ to the summation: Julia follows exactly what you
wrote and adds the elements of `A` to the result `s` in the order it iterates.
Since floating point numbers aren't associative, a rearrangement here would
change the answer — and Julia is loathe to give you different answer than
the one you asked for.

You can, however, tell Julia to relax that rule and allow for associativity
with the `@fastmath` macro. This might allow Julia to rearrange the sum in an
advantageous manner.

In [ ]:
function mysum_fast(A)
    s = 0.0
    for a in A
        @fastmath s += a
    end
    s
end

In [ ]:
j_bench_hand_fast = @benchmark mysum_fast($a)

In [ ]:
mysum_fast(a)

In [ ]:
d["Julia hand-written fast"] = minimum(j_bench_hand_fast.times) / 1e6
d

# 8. Distributed Julia (built-in)

We can take this one step further: nearly every modern computer these days has
multiple cores. All the above solutions are working one core hard, but all the
others are just sitting by idly. Let's put them to work!

In [ ]:
using Distributed
using DistributedArrays
addprocs(4)
@sync @everywhere workers() include("/opt/julia-1.0/etc/julia/startup.jl") # Needed just for JuliaBox
@everywhere using DistributedArrays

In [ ]:
adist = distribute(a)
j_bench_dist = @benchmark sum($adist)

In [ ]:
d["Julia 4x built-in"] = minimum(j_bench_dist.times) / 1e6
d

# 9. Distributed Julia (hand-written)

Ok, that might be cheating, too — it's again just calling a library
function. Is it possible to write distributed sum ourselves?

In [ ]:
function mysum_dist(a::DArray)
    r = Array{Future}(undef, length(procs(a)))
    for (i, id) in enumerate(procs(a))
        r[i] = @spawnat id sum(localpart(a))
    end
    return sum(fetch.(r))
end

In [ ]:
j_bench_hand_dist = @benchmark mysum_dist($adist)

In [ ]:
d["Julia 4x hand-written"] = minimum(j_bench_hand_dist.times) / 1e6
d

# Overall Summary

In [ ]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

# Key take-aways

* Julia allows for serial C-like performance, even with hand-written functions
* Julia allows us to exploit many forms of parallelism to further improve performance. We demonstrated:
    * Single-processor parallelism with SIMD
    * Multi-process parallelism with DistributedArrays
* But there are many other ways to express parallelism, too!